In [43]:
import requests
import json

In [44]:
BASE_URL = "http://localhost:5000"

# Section 1: User Registration

## 1. User Registration

In [45]:
register_data = {
    "username": "test_user",
    "password": "securePass123!"
}
response = requests.post(f"{BASE_URL}/register", json=register_data)
print(f"Status Code: {response.status_code}")
print(json.dumps(response.json(), indent=2))

Status Code: 201
{
  "message": "User created"
}


# Section 2: User Login

## 2. User Login

In [41]:
login_data = {
    "username": "test_user",
    "password": "securePass123!"
}
response = requests.post(f"{BASE_URL}/login", json=login_data)
print(f"Status Code: {response.status_code}")
if response.status_code == 200:
    access_token = response.json()["access_token"]
    print(f"Access Token: {access_token[:15]}...")
else:
    print(json.dumps(response.json(), indent=2))

Status Code: 200
Access Token: eyJhbGciOiJIUzI...


# Section 3: Create Grocery List

## 3. Create Grocery List

In [42]:
headers = {"Authorization": f"Bearer {access_token}"}
list_data = {
    "store_name": "Grocery Store",
    "items": [
        {
            "name": "Milk",
            "price": 2.99
        },
        {
            "name": "Eggs",
            "price": 3.49
        },
        {
            "name": "Bread",
            "price": 2.49
        }
    ]
}
response = requests.post(
    f"{BASE_URL}/lists",
    json=list_data,
    headers=headers
)
print(f"Status Code: {response.status_code}")
if response.status_code == 201:
    result = response.json()
    list_id = result['id']
    print(f"Created List ID: {list_id}")
    print(f"Total: {result['total']}")

Status Code: 201
Created List ID: 1
Total: 8.97


# Section 4: Retrieve Grocery Lists

## 4. Retrieve Grocery Lists

In [27]:
response = requests.get(f"{BASE_URL}/lists", headers=headers)
print(f"Status Code: {response.status_code}")
print("Lists:")
print(json.dumps(response.json(), indent=2))

Status Code: 200
Lists:
[
  {
    "created_at": "2025-03-11T11:46:22",
    "id": 1,
    "item_count": 3,
    "store": "Grocery Store",
    "total": 8.97
  }
]


# Section 5: Update List Test

## 5. Update Grocery List

In [28]:
if lists := response.json():
    list_id = lists[0]['id']
    detail_response = requests.get(
        f"{BASE_URL}/lists/{list_id}",
        headers=headers
    )
    print("Get List Details:")
    print(json.dumps(detail_response.json(), indent=2))

Get List Details:
{
  "id": 1,
  "items": [
    {
      "name": "Milk",
      "price": 2.99,
      "store": "Grocery Store"
    },
    {
      "name": "Eggs",
      "price": 3.49,
      "store": "Grocery Store"
    },
    {
      "name": "Bread",
      "price": 2.49,
      "store": "Grocery Store"
    }
  ],
  "store": "Grocery Store",
  "total": 8.97
}


# Section 6: Verify Update

## 6. Verify Updated List

In [29]:
if lists := response.json():
    list_id = lists[0]['id']
    update_data = {
        "store_name": "Organic Market",
        "items": [
            {
                "name": "Almond Milk",
                "price": 4.99
            },
            {
                "name": "Organic Eggs",
                "price": 5.49
            },
            {
                "name": "Sourdough Bread",
                "price": 3.99
            }
        ]
    }
    update_response = requests.put(
        f"{BASE_URL}/lists/{list_id}",
        json=update_data,
        headers=headers
    )
    print(f"Update Status Code: {update_response.status_code}")
    print("Update Response:")
    print(json.dumps(update_response.json(), indent=2))

Update Status Code: 200
Update Response:
{
  "message": "List updated"
}


# Section 7: Delete List Test

## 7. Delete Grocery List

In [30]:
if lists := verify_response.json():
    list_id = lists[0]['id']
    delete_response = requests.delete(
        f"{BASE_URL}/lists/{list_id}",
        headers=headers
    )
    print(f"Delete Status Code: {delete_response.status_code}")
    print("Delete Response:")
    print(json.dumps(delete_response.json(), indent=2))

Delete Status Code: 200
Delete Response:
{
  "message": "List deleted"
}


# Section 8: Verify Deletion

## 8. Verify List Deletion

In [31]:
final_response = requests.get(f"{BASE_URL}/lists", headers=headers)
print("Remaining Lists:")
print(json.dumps(final_response.json(), indent=2))


Remaining Lists:
[]


In [32]:
verify_response = requests.get(f"{BASE_URL}/lists", headers=headers)
print("Updated Lists:")
print(json.dumps(verify_response.json(), indent=2))

Updated Lists:
[]


In [33]:
stats_response = requests.get(f"{BASE_URL}/stats/monthly", headers=headers)
print("Monthly Stats:")
print(json.dumps(stats_response.json(), indent=2))

Monthly Stats:
[]


# Section 9: Error Case Testing

## 9. Error Case Testing

### Update Non-Existent List

In [34]:
# Test updating non-existent list
response = requests.put(
    f"{BASE_URL}/lists/9999",
    json={"items": [{"name": "test", "price": 1.99}]},
    headers=headers
)
print(f"Status Code: {response.status_code}")
print(json.dumps(response.json(), indent=2))

Status Code: 404
{
  "message": "List not found"
}


### Delete Non-Existent List

In [35]:
response = requests.delete(
    f"{BASE_URL}/lists/9999",
    headers=headers
)
print(f"Status Code: {response.status_code}")
print(json.dumps(response.json(), indent=2))

Status Code: 404
{
  "message": "List not found"
}


### Invalid Login

In [36]:
bad_login = {"username": "wrong_user", "password": "wrong_pass"}
response = requests.post(f"{BASE_URL}/login", json=bad_login)
print(f"Status Code: {response.status_code}")
print(json.dumps(response.json(), indent=2))

Status Code: 401
{
  "message": "Invalid credentials"
}


### Duplicate Registration

In [37]:
# Test invalid item format
response = requests.post(
    f"{BASE_URL}/lists",
    json={"items": ["Invalid item format"]},
    headers=headers
)
print(f"Status Code: {response.status_code}")
print(json.dumps(response.json(), indent=2))

Status Code: 400
{
  "message": "Each item must have name and price"
}


In [38]:
# Test invalid price format
response = requests.post(
    f"{BASE_URL}/lists",
    json={"items": [{"name": "Test", "price": "not-a-number"}]},
    headers=headers
)
print(f"Status Code: {response.status_code}")
print(json.dumps(response.json(), indent=2))

Status Code: 400
{
  "message": "Price must be a number"
}


In [39]:
response = requests.post(f"{BASE_URL}/register", json=register_data)
print(f"Status Code: {response.status_code}")
print(json.dumps(response.json(), indent=2))

Status Code: 400
{
  "message": "Username already exists"
}
